# Qd-Sätze schreiben

In [1]:
import pandas as pd
import re
import numpy as np

Aktuelle csv-Datei wird als df importiert, mit convert_dtypes werden die neuen String-Formate von pandas 1.0 angewendet

In [2]:
df = pd.read_csv('HABV_97_2_v2.csv', index_col='Nr').convert_dtypes()
df.rename({'Verweise/Bemerkungen/Ergänzungen/Zusätze':'Bemerkungen'}, axis=1, inplace=True)
df.drop(['Unnamed: 8', 'Unnamed: 9'], inplace=True, axis=1)
filt = (df['idns'].str.len() <= 2)
df.loc[filt, 'idns'] = np.NaN
df

,Firmenname,Ort,Sortiment,Verlag,Bemerkungen,Lebensdaten,Laufzeit,lz_beginn,lz_ende,firmenname_neu,firmen_matches,idns
Nr,,,,,,,,,,,,
1,aar-Verlag,Wiesbaden,<NA>,x,Jolanda Debus,<NA>,1973-1983,1973,1983,aar-Verlag,"[{'idn': '1065536976', 'level': 'gnd3', 'ort_m...",<NA>
2,A4-Verlag,Essen,<NA>,x,"Thomas Heidelbach, Dieter Sawatzki, Frank Schu...",<NA>,1989-1998,1989,1998,A4-Verlag,[],<NA>
3,Abakon Verlagsgesellschaft,München,<NA>,x,Achim Schneider,<NA>,1976-1987,1976,1987,Abakon Verlagsgesellschaft,"[{'idn': '1065787391', 'level': 'gnd6', 'ort_m...",['1065787391']
4,ABB – Akademischer Buchvertrieb Bonn,"Bad Godesberg, Wesseling",x,<NA>,"Dr. Rudolf Dünnhaupt, Boldt, Athenäum, Junker",<NA>,1955-1986,1955,1986,ABB – Akademischer Buchvertrieb Bonn,[],<NA>
5,ABB Arbeitsgemeinschaft Berliner Buchhändler,Berlin,x,<NA>,Günter Wöckener-Guggisberg,21.06.1949-,1987-1990,1987,1990,ABB Arbeitsgemeinschaft Berliner Buchhändler,[],<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
7137,Zwenka Verlag,Dortmund,<NA>,x,Bernd Wagner,<NA>,1978-1980,1978,1980,Zwenka Verlag,"[{'idn': '1064327141', 'level': 'gnd6', 'ort_m...",['1064327141']
7138,Zwiebelzwerg Company Verlagsgesellschaft,Düsseldorf,<NA>,x,"Gregor Schell, Dieter Herff",<NA>,1981-1986,1981,1986,Zwiebelzwerg Company Verlagsgesellschaft,"[{'idn': '968184545', 'level': 'gnd1', 'ort_ma...",['968184545']
7139,Zwillings-Verlag,Pinneberg,<NA>,x,"Reimer Lübbers, Peter Weidemann",<NA>,1983-1985,1983,1985,Zwillings-Verlag,[],<NA>


In [14]:
def picaplus_schreiben(row):
    pica = f"""001U $0utf8
001X $00
002@ $0Qd
002C $aText$btxt
002D $aohne Hilfsmittel zu benutzen$bn
002E $aBlatt$bnb
010@ $ager
011@ $a{row['lz_beginn']}$b{row['lz_ende']}
011E $r{row['lz_beginn']}-{row['lz_ende']}$4lfzt
013C $aTB-papier
013C $9040445224
013D $9041466098
013E $aa1-analog$aa2-masch$af1-text$af2-blatt$ae2-uv$av-cont
017A $ayy
019@ $aXA-DE
021A $aMitgliedsakte {row['firmenname_neu']}"""
    pica += f"""
021Z $aMitgliedsakte$Bobja
029A $9000073369$BZusammenstellender$4com
033F $p{row.Ort}
034D $a1 Akte
036H $91061153886
"""
    if pd.notna(row.idns):
        idn = re.match(r'\[\'(.+)\'\]', row.idns).group(1)
        pica += f"041A $9{idn}"
    else:
        pica += f"""041A/01 $9000073369
041A/02 $9041146093
041A/09 $eDE-101d$rDE-101d
044N $bRNA$aKorrespondenz
044P $bObjektgattung
044P $9040009254
047A $SBSM
101@ $a1
208@/01 $a12-02-20$bx
201U/01 $0utf8
209A/01 $aHA/BV 97.2,{row.Nr}$dm$x00
209A/01 $fDBSM/S$gHA/BV 97.2,{row.Nr}$x09
245Y/01 $91059758482$8DBSM.Stslg Archiv/BoeHA C 97.2: Unternehmensmitgliedschaft$l{row.Nr}\n\n"""
    return pica

In [ ]:
#jede einzelne zeile des df wird ausgewertet und das ergebnis als string pica zurückgegeben, dieser wird in die geöffnete .dat-Datei geschrieben.

with open('97_2_picaplus-97-2-2021-07-30.dat', 'w') as f:
    for index, row in df.iterrows():
        f.write(picaplus_schreiben(row))

In [3]:
def pica3_schreiben(row, index):
    pica = """\t\n0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
    if pd.isna(row['lz_ende']):
        pica += f"""1100 {row['lz_beginn']}\n1110 *{row['lz_beginn']}$4lfzt"""
    elif pd.notna(row['lz_ende']):
        pica += f"""1100 {row['lz_beginn']}$b{row['lz_ende']}\n1110 *{row['lz_beginn']}-{row['lz_ende']}$4lfzt"""
    pica += """
1130 TB-papier
1130 !040445224!
1130 !041466098!
1132 a1-analog;a2-masch;f1-text;f2-blatt;e2-uv;v-cont
1500 /1ger
1700 /1XA-DE\n"""
    if pd.notna(row.Bemerkungen):
        if match := re.match(r"(.+)(, enth\..+)", row.Bemerkungen):
            pica += f"4000 Mitgliedsakte {row['firmenname_neu']}. {match.group(1)}"
        elif match := re.match(r"^enth.+", row.Bemerkungen):
            pica += f"4000 Mitgliedsakte {row['firmenname_neu']}"
        else:
            pica += f"4000 Mitgliedsakte {row['firmenname_neu']}"
    elif pd.isna(row.Bemerkungen):
        pica += f"4000 Mitgliedsakte {row['firmenname_neu']}"
    else:
        pica += f"4000 Mitgliedsakte {row['firmenname_neu']}"
    pica += f"""
4019 Mitgliedsakte$Bobja
3100 !000073369!$BZusammenstellender$4com"""
    if pd.notna(row.Ort):
        pica += f"\n4046 {row.Ort}"
    pica += """
4060 1 Akte
4105 !1061153886!
5100 !000073369!
"""
    if pd.notna(row.idns):
        if idn := re.match(r"\[\'(\w+)\'\]", row.idns):
            pica += f"5101 !{idn.group(1)}!\n"
        else:
            pica += f"5550 [DBSM-S]{row['firmenname_neu']}\n"
    else:
        pica += f"5550 [DBSM-S]{row['firmenname_neu']}\n"
    pica += "5109 (DE-101d){DE-101d}"
    pica += f"""
5560 [RNA]Korrespondenz
5590 [Objektgattung]
5590 !040009254!
4700 |BSM|
7001 26-11-21 : x
7100 HA/BV 97.2,{index} @ m
7109 !!DBSM/S!! ; HA/BV 97.2,{index}
6710 !1059758482!$l{index}\n"""
    return pica

In [4]:
with open('97_2_pica3-2021-12-02.dat', 'w') as f:
    for index, row in df.iterrows():
        f.write(pica3_schreiben(row, index))

,idn,akten-nr
0,1012288978,HABV_91_1_1634
1,1012291685,HABV_91_1_1749
2,1012562379,HABV_91_1_1176
3,1034719009,HABV_91_1_455
4,104378425X,HABV_91_1_1477
...,...,...
1637,1280506490,HABV_91_1_2883
1638,128134704,HABV_91_1_2588
1639,129414468,HABV_91_1_2099
1640,139313664,HABV_91_1_2356
